In [1]:
import Pkg
include("NN.jl")
using .NN
using RDatasets
using StatsBase

In [2]:
iris = dataset("datasets", "iris");
X = Array(iris[1:4]);
y = iris[5];
n = length(y);

In [3]:
dt = fit(ZScoreTransform, X, dims=1)
X = StatsBase.transform(dt, X)
X = reshape(X, (4, 1, n));

In [4]:
function get_indices(v)
    return CategoricalArrays.order(v.pool)[v.refs]
end
    
refs=get_indices(y);
y=convert(Vector{Int},refs);

In [5]:
y_mod = zeros(UInt8, (n, 3))
for i in 1:n
    y_mod[i, y[i]] = 0x01
end

In [6]:
nn = NeuralNetwork(4, (20,8), 3, η=0.01);

In [7]:
function training(iters=10000)
    costs = zeros(n);
   # nn.learning_rate = 0.000000005 * (10000^2) + 0.0001
    #foo(x) = min(0.000000005x^2 + 0.0001, 0.51)
    for it in 1:iters
        for i in 1:n
            result = train(prepare(X[:, 1, i]'), prepare(y_mod[i, :]'), nn)
            costs[i] = result["cost"]
        end
        #nn.learning_rate = foo(it)
        if it % 100 == 0
            println("error = ", sum(costs))
        end
    end
end

training (generic function with 2 methods)

In [8]:
training()

error = 46.16623678890245
error = 44.27661053027763
error = 43.02495575199823
error = 41.768269150773826
error = 40.43928690106656
error = 39.10126816769174
error = 37.84899068746779
error = 36.73909186244581
error = 35.75466356523181
error = 34.86143854078984
error = 34.04249514678653
error = 33.30132923129583
error = 32.647371363501065
error = 32.06277193204036
error = 31.515484992028156
error = 30.981417963424263
error = 30.446989231410136
error = 29.901517174229422
error = 29.33646796109769
error = 28.74628515494419
error = 28.131667462770828
error = 27.50323619831797
error = 26.876157765341638
error = 26.262805400985158
error = 25.67085073732501
error = 25.103327513962814
error = 24.5596072010199
error = 24.036818326555572
error = 23.531032714951685
error = 23.038126649253712
error = 22.5545942286813
error = 22.078404082131538
error = 21.609490669150315
error = 21.149347552106168
error = 20.699849244935514
error = 20.262069090220418
error = 19.835648821132633
error = 19.4188066159

In [11]:
function check()
    checks = Dict("HIT" => 0, "MISS" => 0)
    for i in 1:n
        if argmax(NN.predict(prepare(X[:, 1, i]'), nn)["result"])[2] == y[i]
            checks["HIT"] += 1
        else
            checks["MISS"] += 1
        end
    end
    println("accuracy = $(checks["HIT"] / 150)%")
end

check (generic function with 1 method)

In [12]:
check()

accuracy = 0.94%
